<a href="https://colab.research.google.com/github/marcosgtavares/P1FDR/blob/main/fastwumpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-sat


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.9 MB/s eta 0:00:00


In [344]:
from pysat.formula import CNF
from pysat.solvers import Solver
from sys import exit
import itertools
import math

def convertCoordinateToSymbol(coordinate: tuple[int, int], symbolType: int) -> int:
  return coordinate[0]*100 + coordinate[1] + symbolType
def convertSymbolToCoordinate(coordinate: int) -> tuple[int, int]:
  strNum = str(abs(coordinate))
  return (int(strNum[1] + strNum[2]), int(strNum[3] + strNum[4]))
  #return (int(abs(coordinate)%10000/100), abs(coordinate)%100)
def getSymbol(coordinate: int) -> int:
  strNum = str(abs(coordinate))
  return int(strNum[0])

wumpusFound = -1 #if wumpus is found remove wumpus related clauses

def addClauses(coordinate: tuple[int, int], visited: list[tuple[int, int]], kb: list[list[int]]): #, kb: list[list[int]]
    #kb[0].extend([convertCoordinateToSymbol(coord) for coord in adjCoordinates(coordinate,visited,[],[])])
    pitClauses = [[-convertCoordinateToSymbol(coordinate, 10000), #BREEZE 10000 PIT 20000
      convertCoordinateToSymbol((coordinate[0]-1, coordinate[1]), 20000), 
      convertCoordinateToSymbol((coordinate[0]+1, coordinate[1]), 20000), 
      convertCoordinateToSymbol((coordinate[0], coordinate[1]-1), 20000),
      convertCoordinateToSymbol((coordinate[0], coordinate[1]+1), 20000)],
      [convertCoordinateToSymbol(coordinate, 10000), -convertCoordinateToSymbol((coordinate[0]-1, coordinate[1]), 20000)],
      [convertCoordinateToSymbol(coordinate, 10000), -convertCoordinateToSymbol((coordinate[0]+1, coordinate[1]), 20000)],
      [convertCoordinateToSymbol(coordinate, 10000), -convertCoordinateToSymbol((coordinate[0], coordinate[1]-1), 20000)],
      [convertCoordinateToSymbol(coordinate, 10000), -convertCoordinateToSymbol((coordinate[0], coordinate[1]+1), 20000)]]

    kb.extend(pitClauses)    
    solver.append_formula(pitClauses)
    
    global wumpusFound
    if wumpusFound == -1:
      solver.append_formula([[-convertCoordinateToSymbol(coordinate, 30000), #STENCH 30000 WUMPUS 40000
        convertCoordinateToSymbol((coordinate[0]-1, coordinate[1]), 40000), 
        convertCoordinateToSymbol((coordinate[0]+1, coordinate[1]), 40000), 
        convertCoordinateToSymbol((coordinate[0], coordinate[1]-1), 40000),
        convertCoordinateToSymbol((coordinate[0], coordinate[1]+1), 40000)],
        [convertCoordinateToSymbol(coordinate, 30000), -convertCoordinateToSymbol((coordinate[0]-1, coordinate[1]), 40000)],
        [convertCoordinateToSymbol(coordinate, 30000), -convertCoordinateToSymbol((coordinate[0]+1, coordinate[1]), 40000)],
        [convertCoordinateToSymbol(coordinate, 30000), -convertCoordinateToSymbol((coordinate[0], coordinate[1]-1), 40000)],
        [convertCoordinateToSymbol(coordinate, 30000), -convertCoordinateToSymbol((coordinate[0], coordinate[1]+1), 40000)]])
    # global wumpusFound
    # if wumpusFound == -1:
    #   kb.extend([[-convertCoordinateToSymbol(coordinate, 30000), #STENCH 30000 WUMPUS 40000
    #   convertCoordinateToSymbol((coordinate[0]-1, coordinate[1]), 40000), 
    #   convertCoordinateToSymbol((coordinate[0]+1, coordinate[1]), 40000), 
    #   convertCoordinateToSymbol((coordinate[0], coordinate[1]-1), 40000),
    #   convertCoordinateToSymbol((coordinate[0], coordinate[1]+1), 40000)],
    #   [convertCoordinateToSymbol(coordinate, 30000), -convertCoordinateToSymbol((coordinate[0]-1, coordinate[1]), 40000)],
    #   [convertCoordinateToSymbol(coordinate, 30000), -convertCoordinateToSymbol((coordinate[0]+1, coordinate[1]), 40000)],
    #   [convertCoordinateToSymbol(coordinate, 30000), -convertCoordinateToSymbol((coordinate[0], coordinate[1]-1), 40000)],
    #   [convertCoordinateToSymbol(coordinate, 30000), -convertCoordinateToSymbol((coordinate[0], coordinate[1]+1), 40000)]])

def adjCoordinates(coordinate: tuple[int, int], visited: list[tuple[int, int]], queue: list[tuple[int, int]], danger: list[tuple[int, int]]) -> list[tuple[int, int]]:
  adjCoord = []
  left = (coordinate[0], coordinate[1]-1)
  up = (coordinate[0]-1, coordinate[1])
  right = (coordinate[0], coordinate[1]+1)
  down = (coordinate[0]+1, coordinate[1])
  
  unitedvqd = visited + queue + danger

  if left not in unitedvqd:
    ##print("left")
    adjCoord.append(left)
  
  if up not in unitedvqd:
    ##print("up")
    adjCoord.append(up)
 
  if right not in unitedvqd:
    ##print("rigth")
    adjCoord.append(right)
 
  if down not in unitedvqd:
    ##print("down")
    adjCoord.append(down)
  
  return adjCoord

solver = Solver()

def uptadeSolver(kb: list[list[int]]):
  global solver
  cnf = CNF(from_clauses=kb)
  solver.delete()
  solver = Solver(bootstrap_with=cnf)
  return solver
def temPoco(coordinate: tuple[int, int]) -> bool:
  global solver
  return not solver.solve(assumptions=[-convertCoordinateToSymbol(coordinate, 20000)])

def naoTemPoco(coordinate: tuple[int, int]) -> bool:
  global solver
  return not solver.solve(assumptions=[convertCoordinateToSymbol(coordinate, 20000)])

def temWumpus(coordinate: tuple[int, int]) -> bool:
  if wumpusFound == coordinate:
    return True
  if wumpusFound != -1:
    return False

  global solver
  return not solver.solve(assumptions=[-convertCoordinateToSymbol(coordinate, 40000)])

def naoTemWumpus(coordinate: tuple[int, int]) -> bool:
  if wumpusFound != -1 and coordinate != wumpusFound:
    return True
  if wumpusFound == coordinate:
    return False

  global solver
  return not solver.solve(assumptions=[convertCoordinateToSymbol(coordinate, 40000)])

def removeWumpusClauses(kb: list[list[int]]):
  range = 0
  while range < len(kb):
    if getSymbol(abs(kb[range][0])) in [3, 4]:
      kb.remove(kb[range])
      range-=1
    range +=1
  uptadeSolver(kb)

lastQueue = -1
actualQueue = 0
def addOneWumpusClauses(adjList: list[tuple[int, int]]):#kb,
  global lastQueue
  global actualQueue
  actualQueue = len(adjList) - 1
  if actualQueue - lastQueue >= 2:
    for combination in itertools.combinations(adjList[lastQueue + 1:], 2):
      solver.add_clause([-convertCoordinateToSymbol(combination[0], 40000), -convertCoordinateToSymbol(combination[1], 40000)])
  
  for adj in adjList[:lastQueue + 1]:
    for adj2 in adjList[lastQueue + 1:]:
      solver.add_clause([-convertCoordinateToSymbol(adj, 40000), -convertCoordinateToSymbol(adj2, 40000)])
  lastQueue = actualQueue
  
start = (3, 3)
def sensor(coordinate: tuple[int, int], coordinate2: tuple[int, int]) -> tuple[bool, list[str]]:
  map = [["N","N","N","B","N","N","N"],
         ["N","S","B","P","B","N","N"],
         ["S","W","S","B","N","N","N"],
         ["N","S","N","N","N","B","N"],
         ["N","B","N","N","B","P","B"],
         ["B","P","B","N","N","B","N"],
         ["N","B","N","N","N","N","N"],
         ["N","N","N","N","N","L","L"],
         ["N","B","N","N","B","P","N"],
         ["B","P","B","N","N","L","L"],
         ["N","B","N","N","N","N","N"],
         ["N","N","B","N","N","N","N"],
         ["N","B","P","B","N","N","N"],
         ["N","N","B","N","N","N","N"],
         ["N","N","N","N","N","N","N"],
         ["N","B","N","N","N","N","N"],
         ["B","P","B","N","N","B","N"],
         ["N","B","N","N","B","P","B"],
         ["N","N","N","N","N","B","N"],
         ["N","N","N","N","N","N","N"],
         ["G","N","N","N","N","N","N"]]
 
  global start
  movement = (coordinate[0] - coordinate2[0], coordinate[1] - coordinate2[1])
  
  if start[0] - movement[0] < 0 or start[0] - movement[0] > 20 or start[1] - movement[1] < 0 or start[1] - movement[1] > 6: #Must change according map size
    #print("impossible")
    return False, [""]
  if map[start[0] - movement[0]][start[1] - movement[1]] == "L":
    #print("L")
    return False, [""]
  
  start = (start[0] - movement[0], start[1] - movement[1])
  
  return True, [map[start[0]][start[1]]]
  
def visitCoordinate(coordinate: tuple[int, int], coordinate2: tuple[int, int], kb: list[list[int]]) -> bool: #, queue: list[tuple[int, int]], visited: list[tuple[int, int]], danger: list[tuple[int, int]]
  possible, sensed = sensor(coordinate, coordinate2)
  
  global wumpusFound
  ##print("possible")
  ##print(start)
  ##print(map[start[0]][start[1]])
  ##print((coordinate, coordinate2))
  #vlist.append(start)
  if not possible:
    solver.add_clause([-convertCoordinateToSymbol(coordinate2, 20000)])
    if wumpusFound == -1:
      solver.add_clause([-convertCoordinateToSymbol(coordinate2, 40000)])
    kb.append([-convertCoordinateToSymbol(coordinate2, 20000)])
    return possible
  if sensed[0] == "B":
    solver.add_clause([convertCoordinateToSymbol(coordinate2, 10000)])
    kb.append([convertCoordinateToSymbol(coordinate2, 10000)])
  else:
    solver.add_clause([-convertCoordinateToSymbol(coordinate2, 10000)])
    kb.append([-convertCoordinateToSymbol(coordinate2, 10000)])
  if wumpusFound == -1 and sensed[0] == "S":
    #print("sS")
    solver.add_clause([convertCoordinateToSymbol(coordinate2, 30000)])
    solver.add_clause([-convertCoordinateToSymbol(coordinate2, 40000)])
  elif wumpusFound == -1:
    solver.add_clause([-convertCoordinateToSymbol(coordinate2, 30000)])
    solver.add_clause([-convertCoordinateToSymbol(coordinate2, 40000)])

  if sensed[0] == "G":
    ##print("VITORIA")
    exit()
  solver.add_clause([-convertCoordinateToSymbol(coordinate2, 20000)])
  kb.append([-convertCoordinateToSymbol(coordinate2, 20000)])
    ##print("visit")
  return possible

def atleastonewumpus():
  clauses = []
  for x in range(0, 99):
    for y in range(0, 99):
      clauses.append(convertCoordinateToSymbol((x ,y), 40000))
  solver.add_clause(clauses)


In [345]:
pos = (50, 50)
visited = []
danger = []
queue = []
visited.append(pos)
queue = adjCoordinates(pos, visited, danger, queue)
uniqueQueue = queue[:]
kb = [[-convertCoordinateToSymbol(pos, 10000)], [-convertCoordinateToSymbol(pos, 20000)]]
solver.append_formula([[-convertCoordinateToSymbol(pos, 10000)], [-convertCoordinateToSymbol(pos, 20000)], [-convertCoordinateToSymbol(pos, 30000)], [-convertCoordinateToSymbol(pos, 40000)]])
addClauses(pos, visited, kb)
counter = 0
reallyStuck = False
while queue:
  ##print(kb)
  #print(queue)
  newpos = queue.pop(0)
  ##print(newpos)
  if temWumpus(newpos): #When wumpus is found, remove wumpus and stench clauses. kb contains only pit and breeze clauses.
    #print("w")
    wumpusFound = newpos
    uptadeSolver(kb)
    solver.add_clause([-convertCoordinateToSymbol(newpos, 20000)])
    danger.append(newpos)
    counter -= 1
  elif temPoco(newpos):
    #print("s")
    solver.add_clause([convertCoordinateToSymbol(newpos, 20000)])
    kb.append([convertCoordinateToSymbol(newpos, 20000)])
    if wumpusFound == -1:
      solver.add_clause([-convertCoordinateToSymbol(newpos, 40000)])
    danger.append(newpos)
    counter -= 1
  elif naoTemWumpus(newpos) and naoTemPoco(newpos) :
    #print("n")
    #pos = moveto(newpos, pos, visited)
    if visitCoordinate(pos, newpos, kb): #, kb
      #print("in")
      addClauses(newpos, visited, kb)
      visited.append(newpos)
      aditionsQueue = adjCoordinates(newpos, visited, queue, danger)
      queue.extend(aditionsQueue)
      uniqueQueue.extend(aditionsQueue)
      pos = newpos
      counter -= 1
  else:
    if wumpusFound == -1:
      addOneWumpusClauses(uniqueQueue)
      #print("nothing")

    counter +=1
    if counter > 30:
      if reallyStuck:
        break
      solver.add_clause([convertCoordinateToSymbol(x, 40000) for x in uniqueQueue])
      counter = 0
      reallyStuck = True
      break
    #print("m")
    queue.append(newpos)

SystemExit: ignored

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [103]:
map = [["N","N","N","B","N","N","N"],
        ["N","N","B","P","B","N","N"],
        ["N","N","N","B","N","N","N"],
        ["N","N","N","N","N","B","N"],
        ["N","B","N","N","B","P","B"],
        ["B","P","B","N","N","B","N"],
        ["N","B","N","N","N","N","N"],
        ["N","N","N","N","N","L","L"],
        ["N","B","N","N","S","W","N"],
        ["B","P","B","N","N","L","L"],
        ["N","B","N","N","N","N","N"],
        ["N","N","B","N","N","N","N"],
        ["N","B","P","B","N","N","N"],
        ["N","N","B","N","N","N","N"],
        ["N","N","N","N","N","N","N"],
        ["N","B","N","N","N","N","N"],
        ["B","P","B","N","N","B","N"],
        ["N","B","N","N","B","P","B"],
        ["N","N","N","N","N","B","N"],
        ["N","N","N","N","N","N","N"],
        ["G","N","N","N","N","N","N"]]
map[8][5]

'W'

In [260]:
import time
tem = 0
for i in range(0, 10000):
  tempo = time.time()
  solver.solve()
  tem += time.time() - tempo

print(tem/10000)

0.0012899203300476075


In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
intersection(visited, danger)

[]

In [ ]:
for x in visited:
  #print(x[0]-47, x[1]-47)

3 3
3 2
2 3
3 1
2 2
3 0
1 2
2 0


In [ ]:
uniquelist = []
uniqueall = []
for x in kb:
  for y in x:
    if convertSymbolToCoordinate(y)[0] > 68 or convertSymbolToCoordinate(y)[0] < 47 or convertSymbolToCoordinate(y)[1] > 53 or convertSymbolToCoordinate(y)[1] < 47:
      if convertSymbolToCoordinate(y) not in uniquelist:
       uniquelist.append(convertSymbolToCoordinate(y))

for x in kb:
  for y in x:
    if abs(y) not in uniqueall:
        
      uniqueall.append(abs(y))
for x in kb:
  for y in x:
    if getSymbol(y) in [3, 4]:
      ##print(y)
len(uniquelist), len(uniqueall)

(45, 330)

In [ ]:
uniqueall

NameError: ignored

In [108]:
wf

(55, 52)

In [49]:
len(visited)

53

In [128]:
danger

[(48, 50),
 (49, 48),
 (51, 52),
 (52, 48),
 (55, 52),
 (56, 48),
 (59, 49),
 (64, 52),
 (63, 48)]

In [ ]:
map = [  ["N","N","N","B","N","N","N"],
         ["N","N","B","P","B","N","N"],
         ["N","P","N","B","N","N","N"],
         ["N","N","N","N","N","S","N"],
         ["N","N","B","N","S","W","S"],
         ["N","B","P","B","N","S","N"],
         ["N","N","B","N","N","N","N"]]
map[3][3]

'P'